In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

dl2025miptml_path = kagglehub.competition_download('dl2025miptml')

print('Data source import complete.')


Импортируем необходимые библиотеки

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
%matplotlib inline
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.models as M
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
import pandas as pd
from sklearn.model_selection import train_test_split


# Загрузим данные

Классы:
```
0 T-shirt/top
1 Trouser
2 Pullover
3 Dress
4 Coat
5 Sandal
6 Shirt
7 Sneaker
8 Bag
9 Ankle boot
```

In [ ]:
base_dir = '/kaggle/input/dl2025miptml/fashion-mnist-testn'
# base_dir = 'data/fashion-mnist-testn'
out_dir = '/kaggle/working'

In [ ]:
df = pd.read_csv(f'{base_dir}/fmnist_train.csv')

df.info()

In [ ]:
df.head()

In [ ]:
# Подсчёт строк, в которых есть хотя бы один пропуск
rows_with_nulls = df.isnull().any(axis=1).sum()
print(rows_with_nulls)  # Выведет количество строк с пропусками

У нас есть одна строка (картинка) с пропусками - просто удалим ее

In [ ]:
df = df.dropna()
df.shape

Оценим сбалансированность данных по меткам

In [ ]:
df['label'].value_counts()

Видим, что данные не сильно но разбалансированы - будем делить датасет на тестовый и валидационный с учетом стратификации

In [ ]:
# Загружаем данные (признаки X и целевую переменную y)
X = df.drop(columns=['label', 'Id'])  # Все колонки, кроме целевой и Id
y = df['label']                # Целевая переменная

# Разделяем данные (70% train, 30% validation)
x_train, x_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.3,        # Доля валидационной выборки (20%)
    random_state=11,      # Фиксируем случайность для воспроизводимости
    stratify=y           # Сохраняет распределение классов (для классификации)
)


In [ ]:
print(x_train.shape)
print(x_val.shape)
print (y_train[:10])
print (y_val[:10])

Переведем датафрейм в нампай массив в ожидаемом моделью виде, представив признаки в виде матрицы 28*28

In [ ]:
x_train = x_train.to_numpy().reshape(-1, 28, 28)
x_val = x_val.to_numpy().reshape(-1, 28, 28)
print(x_train.shape)
print(x_val.shape)

In [ ]:
# one-hot encode для ответов
y_train_oh = keras.utils.to_categorical(y_train, 10)
y_val_oh = keras.utils.to_categorical(y_val, 10)

print(y_train_oh.shape)
print(y_train_oh[:5], y_train[:5])

# Собираем модель

Будем минимизировать кросс-энтропию.

In [ ]:
K.clear_session()

model = M.Sequential()
model.add(L.Conv2D(32, kernel_size=5, strides=1, padding='same', input_shape=(28, 28, 1)))
model.add(L.MaxPool2D())
model.add(L.Conv2D(64, kernel_size=5, strides=1, padding='same'))
model.add(L.MaxPool2D())
model.add(L.Conv2D(128, kernel_size=5, strides=1, padding='same'))
model.add(L.MaxPool2D())
model.add(L.Conv2D(256, kernel_size=5, strides=1, padding='same'))
model.add(L.MaxPool2D())
model.add(L.Conv2D(512, kernel_size=5, strides=1, padding='same'))
model.add(L.Flatten())
model.add(L.Dense(10, activation='softmax'))

In [ ]:
model.summary()

 Итак мы будем обучать модель, у которой свыше 4 млн параметров

In [ ]:
model.compile(
    loss='categorical_crossentropy',  # минимизируем кросс-энтропию
    optimizer='adam',
    metrics=['accuracy']  # выводим процент правильных ответов
)

In [ ]:
# центрируем и нормируем, так сети будет проще учиться
x_train_float = x_train.astype(np.float32) / 255 - 0.5
x_val_float = x_val.astype(np.float32) / 255 - 0.5

Обучаем модель (количество эпох получено экспериментальным путем по лучшему показателю на валидационной выборке

In [ ]:
model.fit(
    x_train_float[:, :, :, np.newaxis],
    y_train_oh,
    batch_size=32,
    epochs=5,
    validation_data=(x_val_float[:, :, :, np.newaxis], y_val_oh)
)

Загружаем тестовый датасет

In [ ]:
test_df = pd.read_csv(f'{base_dir}/fmnist_test.csv')
test_df.info()

Приводим к нужному для работы виду

In [ ]:
test_np = test_df.drop(columns=['Id']).to_numpy().reshape(-1, 28, 28)
ids = test_df['Id'].to_numpy()

Делаем предсказание и выбираем одну метку

In [ ]:
res_raw = model.predict(test_np)
res = tf.argmax(res_raw, axis=1)

Готовим итоговый датафрейм

In [ ]:
res_df = pd.DataFrame({
    'Id': ids,
    'label': res.numpy()
})
res_df.head()

Записываем датафрейм в файл

In [ ]:
res_df.to_csv(f'{out_dir}/submission.csv', index=False)